In [1]:
import pandas, os

In [2]:
input_file = '230117_Arnar.lfq.proteins_From_Gudrun.xlsx'

In [3]:
df = pandas.read_excel(input_file)
proteins = list(df['Accession'])
uniprotids = [element.split('|')[0] for element in proteins]
protein_names = [element.split('|')[1] for element in proteins]
print(len(uniprotids), len(protein_names))

7841 7841


# the data for uniprot ids

In [4]:
id_list = ", ".join(uniprotids)
call_head = "curl --form 'from=\"UniProtKB_AC-ID\"' --form 'to=\"Ensembl\"' --form 'ids=\"Accession, {}\"mi inpu' https://rest.uniprot.org/idmapping/run".format(id_list)
os.system(call_head)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{"jobId":"68464d18e01d7b6ba22dc64596da3e48d28ad510"}

100 67361    0    52  100 67309    114   145k --:--:-- --:--:-- --:--:--  145k


0

In [5]:
!wget https://rest.uniprot.org/idmapping/stream/68464d18e01d7b6ba22dc64596da3e48d28ad510\?format=tsv -O returnedfilea.tsv
dfa = pandas.read_csv('returnedfilea.tsv', sep='\t')
dfa.set_index('To', inplace=True)
dfa.rename(columns={'From':'uniprot'}, inplace=True)
dfa

--2024-10-29 15:55:23--  https://rest.uniprot.org/idmapping/stream/68464d18e01d7b6ba22dc64596da3e48d28ad510?format=tsv
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘returnedfilea.tsv’

returnedfilea.tsv       [   <=>              ] 215.30K   403KB/s    in 0.5s    

2024-10-29 15:55:24 (403 KB/s) - ‘returnedfilea.tsv’ saved [220466]



,uniprot
To,
ENSG00000119335.18,Q01105
ENSG00000035687.10,P30520
ENSG00000088682.14,H3BSJ5
ENSG00000005007.14,Q92900
ENSG00000124641.16,B7ZBQ3
...,...
ENSG00000127603.32,E9PLY5
ENSG00000106462.12,Q15910
ENSG00000105664.11,G3XAP6


# the data for protein ids

In [6]:
id_list = ", ".join(protein_names)
call_head = "curl --form 'from=\"UniProtKB_AC-ID\"' --form 'to=\"Ensembl\"' --form 'ids=\"Accession, {}\"mi inpu' https://rest.uniprot.org/idmapping/run".format(id_list)
os.system(call_head)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{"jobId":"c4e7820458ca9a3c32e07705a18006b97d046bb8"}

100  104k    0    52  100  104k    100   200k --:--:-- --:--:-- --:--:--  200k


0

In [7]:
!wget https://rest.uniprot.org/idmapping/stream/c4e7820458ca9a3c32e07705a18006b97d046bb8\?format=tsv -O returnedfileb.tsv
dfb = pandas.read_csv('returnedfileb.tsv', sep='\t')
dfb.set_index('To', inplace=True)
dfb.rename(columns={'From':'protein_name'}, inplace=True)
dfb

--2024-10-29 15:55:25--  https://rest.uniprot.org/idmapping/stream/c4e7820458ca9a3c32e07705a18006b97d046bb8?format=tsv
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘returnedfileb.tsv’

returnedfileb.tsv       [   <=>              ] 253.72K   429KB/s    in 0.6s    

2024-10-29 15:55:25 (429 KB/s) - ‘returnedfileb.tsv’ saved [259811]



,protein_name
To,
ENSG00000119335.18,SET_HUMAN
ENSG00000035687.10,PURA2_HUMAN
ENSG00000088682.14,H3BSJ5_HUMAN
ENSG00000005007.14,RENT1_HUMAN
ENSG00000124641.16,B7ZBQ3_HUMAN
...,...
ENSG00000127603.32,E9PLY5_HUMAN
ENSG00000106462.12,EZH2_HUMAN
ENSG00000105664.11,G3XAP6_HUMAN


# join and store

In [8]:
dfc = dfa.merge(dfb, left_on="To", right_on="To", how='outer')
dfc

,uniprot,protein_name
To,,
ENSG00000000003.16,O43657,TSN6_HUMAN
ENSG00000000419.14,O60762,DPM1_HUMAN
ENSG00000000419.14,O60762,Q5QPK2_HUMAN
ENSG00000000419.14,O60762,H0Y368_HUMAN
ENSG00000000419.14,Q5QPK2,DPM1_HUMAN
...,...,...
ENSG00000292348.1,A0A096LP69,CD99_HUMAN
ENSG00000292348.1,P14209,A0A096LP69_HUMAN
ENSG00000292348.1,P14209,CD99_HUMAN


In [9]:
dfc.loc['ENSG00000088682.14', :]

,uniprot,protein_name
To,,
ENSG00000088682.14,H3BSJ5,H3BSJ5_HUMAN
ENSG00000088682.14,H3BSJ5,COQ9_HUMAN
ENSG00000088682.14,O75208,H3BSJ5_HUMAN
ENSG00000088682.14,O75208,COQ9_HUMAN


In [10]:
dfc.index

Index(['ENSG00000000003.16', 'ENSG00000000419.14', 'ENSG00000000419.14',
       'ENSG00000000419.14', 'ENSG00000000419.14', 'ENSG00000000419.14',
       'ENSG00000000419.14', 'ENSG00000000419.14', 'ENSG00000000419.14',
       'ENSG00000000419.14',
       ...
       'ENSG00000292253.1', 'ENSG00000292334.1', 'ENSG00000292338.1',
       'ENSG00000292339.1', 'ENSG00000292348.1', 'ENSG00000292348.1',
       'ENSG00000292348.1', 'ENSG00000292348.1', 'ENSG00000292358.1',
       'ENSG00000292366.1'],
      dtype='object', name='To', length=16061)

In [11]:
new_index = [element.split('.')[0] for element in dfc.index]
dfc['ensembl'] = new_index
dfc.set_index('ensembl', inplace=True)

In [12]:
dfc.to_csv('idmap.tsv', sep = '\t')